In [1]:
import numpy as np

In [2]:
import katarchive

import datetime

import logging
logging.root.setLevel(logging.ERROR)


In [3]:
#today=datetime.datetime.today()
#ago=datetime.timedelta(-14)   #  days is a fortnight
#lastweek=today+ago
#print today,lastweek
#begin=lastweek.strftime(format="%d/%m/%Y")
begin="30/06/2016"
#end=today.strftime(format="%d/%m/%Y")
end="31/07/2016"

In [4]:
def findh5(det):   # uses detail list to get h5 name
    pos=det.find("Name:")
    if pos >0  :
        pr2=det[pos+37:pos+53].split()
        h5name=pr2[0].lstrip("/ng")
    return h5name

In [5]:
def findproj(text):
    found='Unknown'        # default
    chx=text.find('prop')
    if chx >0  :
        pr2=text[chx+11:chx+38].split()
        code=pr2[0]
        found=code.replace("=","")
    else:
        if text.find('maser') >0:
            found="Maser monitoring"
            #print text
    return found

In [6]:
def listpr(x):
    """
    takes a list and prints as string
    """
    li=" ".join(map(str,x))
    return li

In [7]:
f = katarchive.search_archive(startdate=begin,enddate=end)
start = datetime.datetime.strptime(f[-1].__str__().split('\'')[3],'%Y-%m-%d %H:%M:%S %Z')
end = datetime.datetime.strptime(f[0].__str__().split('\'')[3],'%Y-%m-%d %H:%M:%S %Z')

Searched using: CAS.ProductTypeName:(KatFile) AND CAS.ProductTransferStatus:RECEIVED AND StartTime:[2016-06-29T22:00:00Z TO 2016-07-31T21:59:59Z]
Returning 151 results


In [8]:
#print 'First file in run: ' , start.isoformat()
#print 'Last file in run: ' , end.isoformat()

In [9]:
for fi in f:
    txts=fi.metadata.InstructionSet
    p=findproj(txts)
    print fi.metadata.ExperimentID, p, fi.metadata.Duration, fi.metadata.Description, len(fi.metadata.Antennas)


2 COMM_Atten 109.01 Auto atten 5
2 COMM_Image_NO 43980.61 Imaging Calibration 5
2 COMM_Atten 109.01 Auto atten 5
2 COMM_Image_NO 18070.25 Imaging Calibration 5
2 COMM_Atten 78.01 Auto atten 5
2 Unknown 595.05 Beamformer observation 5
2 Unknown 567.04 Beamformer observation 5
2 COMM_Image_NO 6530.09 Imaging Calibration 5
2 COMM_Atten 42.0 Auto atten 5
2 COMM_Atten 108.01 Auto atten 5
2 COMM_RFI_SP 5976.47 Basic RFI Scan: RFI SCAN 5
2 COMM_Atten 108.01 Auto atten 5
2 COMM_Atten 139.01 Auto atten 5
2 20150619TF-01 18050.25 3C279 5
2 COMM_Atten 108.01 Auto atten 5
2 COMM_Image_NO 7460.1 Imaging Calibration 5
2 COMM_Atten 86.01 Auto atten 5
2 20130822NO-01 28960.4 W2332_5056 5
2 COMM_Atten 110.01 Auto atten 5
2 COMM_Image_NO 24330.34 Imaging Calibration 5
2 COMM_Atten 75.01 Auto atten 5
2 Unknown 626.05 Beamformer observation 5
2 COMM_Image_NO 15070.21 Imaging Calibration 5
2 COMM_Atten 93.01 Auto atten 5
2 COMM_RFI_SP 5974.47 Basic RFI Scan: RFI SCAN 5
2 COMM_Atten 41.0 Auto atten 5
2 COMM

Only select observations that are longer than 1 hrs then extract the project id from the script parameters. If there is no project id then allocated No_project_id 

In [10]:
sbd = [] #Experiment ID
pid = [] #project ID
fnd = [] #Filename
dud = [] #Duration
antsd = [] #number of antennas
scrd = [] #script  -NOT USED NOW

for fi in f:
    if fi.metadata.Duration < 3600: #only pick hour long or greater observations for analysis
        continue
    # now try extract the project-id from the instruction-set string
    """
    script = fi.metadata.InstructionSet
    scrd.append(script)
    spl = script.split()
    """
    txts=fi.metadata.InstructionSet
    p=findproj(txts)
    if p == "Unknown":
        p="Maser Monitoring"
    sbd.append(fi.metadata.ExperimentID)
    pid.append(p)
    fnd.append(fi.metadata.Filename)
    dud.append(fi.metadata.Duration)
    antsd.append(len(fi.metadata.Antennas))


In [11]:
pi = np.array(pid)
sb = np.array(sbd)
du = np.array(dud)
ants = np.array(antsd)

p_cent = (sum(du)/((end.toordinal()-start.toordinal())*24*3600))*100

print 'Percentage system usage: ', np.floor(p_cent),'%'


Percentage system usage:  58.0 %


In [12]:
a_id = []
a_time = []
for p in set(pi):
    ind = pi == p
    print p,round(sum(du[ind])/3600,2)
    a_id.append(p)
    a_time.append(round(sum(du[ind])/3600,2))

20141111IM-01 4.11
COMM_Tipping_NO 1.07
20130822NO-01 23.32
COMM_Bandpass_TF 24.31
20150619TF-01 15.2
20141219SS-01 11.39
COMM_RFI_SP 17.91
COMM_Image_NO 225.74
COMM_Point_LM 5.13
20131211NO-01 3.04
Maser monitoring 31.96


In [13]:
ind = np.argsort(a_id)
for i in ind:
    print ("%-16s %5.1f %d") %(a_id[i],a_time[i],ants[i])

20130822NO-01     23.3 5
20131211NO-01      3.0 5
20141111IM-01      4.1 5
20141219SS-01     11.4 5
20150619TF-01     15.2 5
COMM_Bandpass_TF  24.3 5
COMM_Image_NO    225.7 5
COMM_Point_LM      5.1 5
COMM_RFI_SP       17.9 5
COMM_Tipping_NO    1.1 5
Maser monitoring  32.0 5


Find the unique project ids and determine the total observation duration.

Sort ID and time for display

In [14]:
for fi in f:
    x= repr(fi)
    pr=x.split(",")
    print pr[1][10:12],findh5(fi.metadata.Details) , fi.metadata.Description, listpr(fi.metadata.Antennas)

26 1469518626.h5 Auto atten ant1 ant2 ant3 ant5 ant6
25 1469471407.h5 Imaging Calibration ant1 ant2 ant3 ant5 ant6
25 1469471170.h5 Auto atten ant1 ant2 ant3 ant5 ant6
25 1469452203.h5 Imaging Calibration ant1 ant2 ant3 ant5 ant6
25 1469452067.h5 Auto atten ant1 ant2 ant3 ant5 ant6
25 1469450387.h5 Beamformer observation ant1 ant2 ant3 ant5 ant6
25 1469447953.h5 Beamformer observation ant1 ant2 ant3 ant5 ant6
25 1469440026.h5 Imaging Calibration ant1 ant2 ant3 ant5 ant6
25 1469439826.h5 Auto atten ant1 ant2 ant3 ant5 ant6
25 1469439596.h5 Auto atten ant1 ant2 ant3 ant5 ant6
25 1469431714.h5 Basic RFI Scan: RFI SCAN ant1 ant2 ant3 ant5 ant6
25 1469431560.h5 Auto atten ant1 ant2 ant3 ant5 ant6
23 1469310974.h5 Auto atten ant1 ant2 ant3 ant5 ant6
23 1469268009.h5 3C279 ant1 ant2 ant3 ant5 ant6
23 1469267770.h5 Auto atten ant1 ant2 ant3 ant5 ant6
23 1469257606.h5 Imaging Calibration ant1 ant2 ant3 ant5 ant6
23 1469257485.h5 Auto atten ant1 ant2 ant3 ant5 ant6
23 1469228476.h5 W2332_5056 an

In [15]:
f.sort(key=lambda k: k.metadata.StartTime) #Sort in ascending time+++++++++++++++++++++++++++++

In [16]:
print "Filename\t\t\t\t\t\t Project \tTitle"
for fi in f:
    x= repr(fi)
    pr=x.split(",")
    nrant=len(fi.metadata.Antennas) # number of antennas
    proj=findproj(fi.metadata.InstructionSet)
    if fi.metadata.Description.find('Auto') <0 and  fi.metadata.Description.find('RFI') <0 and \
      fi.metadata.Description.find('Pointing') <0  and  fi.metadata.Description.find('Delay') <0  and \
      fi.metadata.Description.find('Tipping') <0  and nrant>4 :  # not auto attenuate, more than 5 antenna
        print("%s/%s  %-18s %s") %(fi.metadata.FileLocation,findh5(fi.metadata.Details),proj,fi.metadata.Description )

Filename						 Project 	Title
/var/kat/archive/data/comm/2016/07/05/1467749409.h5  Maser monitoring   G9.62 0.20 daily monitoring
/var/kat/archive/data/comm/2016/07/06/1467835211.h5  COMM_Image_NO      Imaging Calibration
/var/kat/archive/data/comm/2016/07/08/1467957907.h5  COMM_Image_NO      Imaging Calibration
/var/kat/archive/data/comm/2016/07/08/1467974170.h5  Unknown            Target track
/var/kat/archive/data/comm/2016/07/08/1467974832.h5  Unknown            Beamformer observation
/var/kat/archive/data/comm/2016/07/08/1467990453.h5  COMM_Image_NO      Imaging Calibration
/var/kat/archive/data/comm/2016/07/09/1468055910.h5  COMM_Image_NO      Imaging Calibration
/var/kat/archive/data/comm/2016/07/10/1468120926.h5  COMM_Image_NO      Imaging Calibration
/var/kat/archive/data/comm/2016/07/10/1468185984.h5  COMM_Image_NO      Imaging Calibration
/var/kat/archive/data/comm/2016/07/11/1468251091.h5  COMM_Image_NO      Imaging Calibration
/var/kat/archive/data/comm/2016/07/12/14683274